daily_run estimates strikes, number of contracts for a put selling strategy on S&P 500 based on "Cost of Capital for Alternatives" - Data requires either an open session of IB Gateway or TWS from Inter-active brokers

In [1]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
import numpy as np
import pandas as pd
import nest_asyncio
from time import time

before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [5]:
z_score = np.array([-1, -1.5, -2])
leverage = np.array(range(1, 4, 1))
num_expiries = 3
illiquid_equity = 500000
after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 51.9
elapsed: 75.0
2632.56
21.52
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2701789.6823', currency='BASE', modelCode='')]


In [6]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [7]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-1.0,-1.5,-2.0
2018.12.21,2490,2425,2380
2019.01.18,2420,2330,2265
2019.02.15,2355,2235,2150


,-1.0,-1.5,-2.0
2018.12.21,12.35,7.00,4.85
2019.01.18,18.20,10.85,7.55
2019.02.15,22.85,13.25,9.00


,-1.0,-1.5,-2.0
2018.12.21,12.0/12.7,6.7/7.3,4.6/5.1
2019.01.18,17.8/18.6,10.6/11.1,7.3/7.8
2019.02.15,22.5/23.2,13.0/13.5,8.8/9.2


In [8]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-1.0,-1.5,-2.0
2018.12.21,-5.42%,-7.88%,-9.59%
2019.01.18,-8.07%,-11.49%,-13.96%
2019.02.15,-10.54%,-15.10%,-18.33%


In [9]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-1.0,-1.5,-2.0
2018.12.21,"$16,055","$9,100","$6,790"
2019.01.18,"$23,660","$15,190","$10,570"
2019.02.15,"$31,990","$18,550","$13,500"


,-1.0,-1.5,-2.0
2018.12.21,"$32,110","$18,900","$13,095"
2019.01.18,"$49,140","$30,380","$21,140"
2019.02.15,"$63,980","$38,425","$27,000"


,-1.0,-1.5,-2.0
2018.12.21,"$48,165","$28,000","$19,885"
2019.01.18,"$72,800","$45,570","$32,465"
2019.02.15,"$93,685","$56,975","$40,500"


In [10]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-1.0,-1.5,-2.0
2018.12.21,0.47%,0.27%,0.18%
2019.01.18,0.69%,0.41%,0.29%
2019.02.15,0.87%,0.50%,0.34%


,-1.0,-1.5,-2.0
2018.12.21,0.94%,0.53%,0.37%
2019.01.18,1.38%,0.82%,0.57%
2019.02.15,1.74%,1.01%,0.68%


,-1.0,-1.5,-2.0
2018.12.21,1.41%,0.80%,0.55%
2019.01.18,2.07%,1.24%,0.86%
2019.02.15,2.60%,1.51%,1.03%


In [11]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,13,13,14
2019.01.18,13,14,14
2019.02.15,14,14,15


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,26,27,27
2019.01.18,27,28,28
2019.02.15,28,29,30


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,39,40,41
2019.01.18,40,42,43
2019.02.15,41,43,45


In [12]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-1.0,-1.5,-2.0
2018.12.21,"$515,193","$423,738","$390,323"
2019.01.18,"$431,798","$341,390","$327,670"
2019.02.15,"$380,523","$331,450","$336,000"


,-1.0,-1.5,-2.0
2018.12.21,"$1,030,385","$880,070","$752,765"
2019.01.18,"$896,810","$682,780","$655,340"
2019.02.15,"$761,046","$686,575","$672,000"


,-1.0,-1.5,-2.0
2018.12.21,"$1,545,578","$1,303,808","$1,143,088"
2019.01.18,"$1,328,608","$1,024,170","$1,006,415"
2019.02.15,"$1,114,388","$1,018,025","$1,008,000"


In [13]:
marge = trd_choice.margin(1200)
display(marge.style.format('${:,.0f}'))

,-1.0,-1.5,-2.0
2018.12.21,"$154,235","$147,200","$142,485"
2019.01.18,"$147,820","$138,085","$131,255"
2019.02.15,"$141,785","$128,825","$119,900"


In [14]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,"$3,237,000","$3,152,500","$3,332,000"
2019.01.18,"$3,146,000","$3,262,000","$3,171,000"
2019.02.15,"$3,297,000","$3,129,000","$3,225,000"


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$6,474,000","$6,547,500","$6,426,000"
2019.01.18,"$6,534,000","$6,524,000","$6,342,000"
2019.02.15,"$6,594,000","$6,481,500","$6,450,000"


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,"$9,711,000","$9,700,000","$9,758,000"
2019.01.18,"$9,680,000","$9,786,000","$9,739,500"
2019.02.15,"$9,655,500","$9,610,500","$9,675,000"
